In [1]:
import pandas as pd
import numpy as np
import os
import re
from collections import Counter
import math

In [2]:
# new_item_codes=pd.read_excel("C:\\Users\\Geoffrey.Oute\\Desktop\\Automation Sandbox\\SAP Old_New Item Codes.xlsx")

In [3]:
selling_price=pd.read_excel("C:\\Users\\Geoffrey.Oute\\Desktop\\Automation Sandbox\\Retail Label Printing Inc. VAT_04_05.xlsx")

In [4]:
Tier1Shops=pd.DataFrame({'Store/Location Name': ['RETAIL SARIT', 'RETAIL YAYA', 'RETAIL HILTON', 'RETAIL VILLAGE MARKET'], 'Tier': ['Tier1', 'Tier1', 'Tier1', 'Tier1']})

In [215]:
blocked_items = pd.read_excel("C:\\Users\\Geoffrey.Oute\\Desktop\\Automation Sandbox\\Item Block.xlsx")

In [216]:
blocked_items.head(2)

,Item No.,Retail Shop
0,101000098,RETAIL REHANI
1,137000148,RETAIL REHANI


In [7]:
Tier1Shops

,Store/Location Name,Tier
0,RETAIL SARIT,Tier1
1,RETAIL YAYA,Tier1
2,RETAIL HILTON,Tier1
3,RETAIL VILLAGE MARKET,Tier1


In [8]:
stock_and_sales=pd.read_csv("C:\\Users\\Geoffrey.Oute\\Desktop\\Automation Sandbox\\Stock&Monthwise_2022TD_Stock+Sales.csv")

C:\Users\Geoffrey.Oute\AppData\Local\Temp\ipykernel_17320\2198782836.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  stock_and_sales=pd.read_csv("C:\\Users\\Geoffrey.Oute\\Desktop\\Automation Sandbox\\Stock&Monthwise_2022TD_Stock+Sales.csv")


In [9]:
GP_table=pd.read_csv("C:\\Users\\Geoffrey.Oute\\Desktop\\Automation Sandbox\\GPs.csv")

C:\Users\Geoffrey.Oute\AppData\Local\Temp\ipykernel_17320\1838810347.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  GP_table=pd.read_csv("C:\\Users\\Geoffrey.Oute\\Desktop\\Automation Sandbox\\GPs.csv")


In [10]:
stock_and_salesB=stock_and_sales.copy()

In [11]:
stock_and_sales.head(2)

,index,Item No.,Item Description,Brand,Department,Location Type,Store/Location Name,Month_selling_frequency,Monthly_selling_frequency_last6months,Monthly_selling_frequency_last3months,...,Jan Qty Sold2024,Jan Stock2024,month12024sales,month12024COGS,Jan2024SellRate,Feb Qty Sold2024,Feb Stock2024,month22024sales,month22024COGS,Feb2024SellRate
0,0,167003170,"""""CANDLELIGHT 5X7"""" RESIN PHOTO FRAME LOOPS GO...",CANDLELIGHT,Giftware,Retail,RETAIL THE FIVE ELEMENTS,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,31429000005,"""""CWS CREATE YOUR OWN DINOSAUR TIMELINE & WORL...",CWS,Giftware,Warehouse,MAIN WAREHOUSE,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
x=12
y=13
z=14
for i in [i for i in stock_and_salesB if 'Sold' in i]:
    GP=(stock_and_salesB.iloc[:,x]-stock_and_salesB.iloc[:,y])/stock_and_salesB.iloc[:,x]
    colname=stock_and_salesB.iloc[:, y].name.replace('COGS', 'GP')
    stock_and_salesB.insert(z, colname, GP)
    x+=6
    y+=6
    z+=6

In [13]:
stock_and_salesB.columns

Index(['index', 'Item No.', 'Item Description', 'Brand', 'Department',
       'Location Type', 'Store/Location Name', 'Month_selling_frequency',
       'Monthly_selling_frequency_last6months',
       'Monthly_selling_frequency_last3months', 'Jan Qty Sold2023',
       'Jan Stock2023', 'month12023sales', 'month12023COGS', 'month12023GP',
       'Jan2023SellRate', 'Feb Qty Sold2023', 'Feb Stock2023',
       'month22023sales', 'month22023COGS', 'month22023GP', 'Feb2023SellRate',
       'Mar Qty Sold', 'Mar Stock', 'month32023sales', 'month32023COGS',
       'month32023GP', 'Mar2023SellRate', 'Apr Qty Sold', 'Apr Stock',
       'month42023sales', 'month42023COGS', 'month42023GP', 'Apr2023SellRate',
       'May Qty Sold', 'May Stock', 'month52023sales', 'month52023COGS',
       'month52023GP', 'May2023SellRate', 'Jun Qty Sold', 'Jun Stock',
       'month62023sales', 'month62023COGS', 'month62023GP', 'Jun2023SellRate',
       'Jul Qty Sold', 'Jul Stock', 'month72023sales', 'month72023COGS',
 

In [14]:
gps_array=stock_and_salesB [[i for i in stock_and_salesB.columns if 'GP' in i]].values

In [15]:
sorted_gps_array=-np.sort(-gps_array)

In [16]:
largest_gps=sorted_gps_array[:, :4]

In [17]:
maxGP=np.nanmedian(largest_gps, axis=1)

C:\Users\Geoffrey.Oute\AppData\Local\Temp\ipykernel_17320\3354881009.py:1: RuntimeWarning: All-NaN slice encountered
  maxGP=np.nanmedian(largest_gps, axis=1)


In [18]:
final=pd.concat([stock_and_salesB, pd.DataFrame(maxGP)], axis=1)

In [19]:
final.rename(columns={0:'OptimumGP'}, inplace=True)

In [20]:
gp_db=final[['Item Description', 'Item No.', 'Store/Location Name', 'OptimumGP']]

In [21]:
a=12
b=10
c=14
for i in [i for i in stock_and_salesB if 'Sold' in i]: # change here to final instead of stock and sales
    unit_cost=final.iloc[:,a]/final.iloc[:,b]
    col_name=final.iloc[:, a].name.replace('sales', 'unit_cost')
    final.insert(c, col_name, unit_cost)
    a+=7
    b+=7
    c+=7

In [22]:
price_brackets_table = pd.DataFrame(columns=['0-200', '200-400', '400-600', '600-800', '800-1000', 
                                             '1000-1500', '1500-2500', '2500+'])

In [23]:
price_brackets_table

,0-200,200-400,400-600,600-800,800-1000,1000-1500,1500-2500,2500+


In [24]:
final_final=pd.concat([final, price_brackets_table], axis=1)

In [25]:
filled_0=final_final.loc[:, '0-200':].fillna(0)

In [26]:
final_final.drop([i for i in final_final.columns if '00' in i], inplace=True, axis=1)

In [27]:
final_final_final=pd.concat([final, filled_0], axis=1)

In [28]:
final_final_final=pd.concat([final_final_final.loc[:, 'index': 'Store/Location Name'], final_final_final.loc[:, 'Dec Qty Sold2022':]], axis=1)

KeyError: 'Dec Qty Sold2022'

In [ ]:
final_final_final.iloc[:, 7:]

In [ ]:
s=12
e=13
q=8

#mask if unit_price>0 but less than 200 and gp was above 0.23
#q is column for quantity sold. .iloc.name so that we can get the name of the column. mask doesnt work iloc; only loc.
for i in [i for i in final_final_final if 'Sold' in i]:
    mask1=(final_final_final.iloc[:, s] > 0) & (final_final_final.iloc[:, s]<200) & (final_final_final.iloc[:, e]>0.23)
    final_final_final.loc[mask1, '0-200']+=final_final_final.loc[mask1, final_final_final.iloc[:, q].name]
    
    mask2=(final_final_final.iloc[:, s] >= 200) & (final_final_final.iloc[:, s]<400) & (final_final_final.iloc[:, e]>0.23)
    final_final_final.loc[mask2, '200-400']+=final_final_final.loc[mask2, final_final_final.iloc[:, q].name]
    
    mask3=(final_final_final.iloc[:, s] >= 400) & (final_final_final.iloc[:, s]<600) & (final_final_final.iloc[:, e]>0.23)
    final_final_final.loc[mask3, '400-600']+=final_final_final.loc[mask3, final_final_final.iloc[:, q].name]

    mask4=(final_final_final.iloc[:, s] >= 600) & (final_final_final.iloc[:, s]<800) & (final_final_final.iloc[:, e]>0.23)
    final_final_final.loc[mask4, '600-800']+=final_final_final.loc[mask4, final_final_final.iloc[:, q].name]
    
    mask5=(final_final_final.iloc[:, s] >= 800) & (final_final_final.iloc[:, s]<1000) & (final_final_final.iloc[:, e]>0.23)
    final_final_final.loc[mask5, '800-1000']+=final_final_final.loc[mask5, final_final_final.iloc[:, q].name]
    
    mask6=(final_final_final.iloc[:, s] >= 1000) & (final_final_final.iloc[:, s]<1500) & (final_final_final.iloc[:, e]>0.23)
    final_final_final.loc[mask6, '1000-1500']+=final_final_final.loc[mask6, final_final_final.iloc[:, q].name]
    
    mask7=(final_final_final.iloc[:, s] >= 1500) & (final_final_final.iloc[:, s]<2500) & (final_final_final.iloc[:, e]>0.23)
    final_final_final.loc[mask7, '1500-2500']+=final_final_final.loc[mask7, final_final_final.iloc[:, q].name]
    
    mask8=(final_final_final.iloc[:, s] >= 2500) & (final_final_final.iloc[:, e]>0.23)
    final_final_final.loc[mask8, '2500+']+=final_final_final.loc[mask8, final_final_final.iloc[:, q].name]
    
        
    
    s+=7
    e+=7
    q+=7
    

In [29]:
# pd.concat([final_final_final.iloc[:, :8], final_final_final.loc[:, '0-200':]], axis=1).to_csv('price_analysis_table_Dec2022tonow.csv')
 # table to analyze   

In [30]:
#final_final_final.iloc[:, :8]

In [31]:
final_final_final.head()

,index,Item No.,Item Description,Brand,Department,Location Type,Store/Location Name,Month_selling_frequency,Monthly_selling_frequency_last6months,Monthly_selling_frequency_last3months,...,Feb2024SellRate,OptimumGP,0-200,200-400,400-600,600-800,800-1000,1000-1500,1500-2500,2500+
0,0,167003170,"""""CANDLELIGHT 5X7"""" RESIN PHOTO FRAME LOOPS GO...",CANDLELIGHT,Giftware,Retail,RETAIL THE FIVE ELEMENTS,0,0,0,...,0.0,NaN,0,0,0,0,0,0,0,0
1,1,31429000005,"""""CWS CREATE YOUR OWN DINOSAUR TIMELINE & WORL...",CWS,Giftware,Warehouse,MAIN WAREHOUSE,1,1,1,...,0.0,0.102615,0,0,0,0,0,0,0,0
2,2,167002159,"""""ELYSIUM AVOCADO, LIME AND SEA SALT SCRUB 200...",STRAITS,Giftware,Warehouse,GIFTWARE WAREHOUSE,0,0,0,...,0.0,NaN,0,0,0,0,0,0,0,0
3,3,167002160,"""""ELYSIUM TEA TREE, LEMON AND SEA SALT SCRUB 2...",STRAITS,Giftware,Warehouse,GIFTWARE WAREHOUSE,0,0,0,...,0.0,NaN,0,0,0,0,0,0,0,0
4,4,31429450011,"""""EUROWRAP 18"""" GONK CUTE FOIL BALLOON 48S #X-...",EUROWRAP,Giftware,Warehouse,GIFTWARE WAREHOUSE,0,0,0,...,0.0,NaN,0,0,0,0,0,0,0,0


OKAY. NOW WE START THE ROTATION PROCESS. PRICES ANALYSIS WILL PROVE USEFUL WHEN IT COMES TO PRODUCTS THAT CAN ROTATE BASED ON HISTORICAL SALES. WE'LL HAVE TO LOOK AT IT FROM A DEPARTMENTAL POV

In [32]:
#stock_and_sales [stock_and_sales ['Item Description'] == "MARIGOLD VEGAN GF ENGEVITA YEAST FLAKES 100G#MG15689"]

In [33]:
stock_and_sales_C=stock_and_sales.copy()

In [34]:
stock_and_sales_C['Item No.']=stock_and_sales_C ['Item No.'].astype(str)

In [35]:
#Omitting Carrefour

In [36]:
stock_and_sales_C=stock_and_sales_C[~stock_and_sales_C.isin({'Store/Location Name': ['CARREFOUR MEGA (EXPRESS)']})['Store/Location Name']]

In [37]:
stock_and_sales_C=stock_and_sales_C[~stock_and_sales_C.isin({'Department': ['Giftware', 'Gftware', 'Packaging Materials', 'Assets & Consumables']})['Department']]

We need to create a move to table, based on sales and GP
If GP is above 0.23 and sell rate is 0.25 and shop can hold the incoming stock for the next two months. Shop transferring the item should be left with two months of sales stock.

Rotate to shops first, then to warehouse if item is shared. How will we know shared items? Anyhting that has sold in both retail and main can be considered as shared. However, if it hasn't sold in main for the last 3 months, we can assume it has switched to retail only and vice versa.

In [38]:
stock_shops=stock_and_sales_C[['Item Description', 'Store/Location Name']].groupby('Item Description')['Store/Location Name'].agg(set)

In [39]:
stock_shop_dict=stock_shops.to_dict()

In [40]:
shared=[]
for i in stock_shop_dict:
    if 'MAIN WAREHOUSE' in stock_shop_dict[i]:
        shared.append(i)

shared has a list of all shared items

In [41]:
Instock_import = pd.read_excel("C:\\Users\\Geoffrey.Oute\\Desktop\\David\\Stock & Monthwise Current Month\\03_2024.xlsx")

In [42]:
Instock_import ['Item No.']= Instock_import ['Item No.'].astype(str)

In [43]:
path_current_month="C:\\Users\\Geoffrey.Oute\\Desktop\\David\\Stock & Monthwise Current Month"
file_names_current_month=[]
for i in os.listdir(path_current_month):
    file_names_current_month.append(i.replace('.xlsx', ""))


In [44]:
file_names_current_month

['03_2024']

In [45]:
#DYMATIZE ISO-100 GOURMET VANILLA 2.3KG #D35321A

In [46]:
#Lets calculate current month rate of sale. Will help to check if the sale is picking up. If it is picking up, then we don't
# move the item. If it is constant or dropping, then transfer.

index_start=0

index_trace=[]
holding_list=[]
for i in file_names_current_month:
    month = re.search('[0-9]+', i).group()
    year=re.search('[^_]*$', i).group()
    
    file_name="C:\\Users\\Geoffrey.Oute\\Desktop\\David\\Stock & Monthwise Current Month\\{x}_{y}.xlsx"
    temp_table=pd.read_excel(file_name.format(x=month, y=year))
    temp_table['Month']= int(month)
    temp_table['Year']=int(year)
    index_trace.append(len(temp_table))
    temp_table.set_axis(range(index_start, sum(index_trace)), inplace=True)
    temp_table['monthlystock'+month]= temp_table['Total Quantity Sold']+temp_table['Closing Stock']
    temp_table['month'+month+year+'sales']=temp_table['Total Sales Amount']
    #temp_table['month'+month+year+'GP']=temp_table['Gross Profit %']
    temp_table ['month'+month+year+'COGS']=temp_table['COGS']
    holding_list.append(temp_table)
    #print (index_start, index_trace )
    index_start=sum(index_trace)
    
table_current_month=pd.concat(holding_list)

In [48]:
QtySold=table_current_month[[ i for i in table_current_month.columns if 'Qty' in i ]].sum(axis=1)
table_current_month['Current_Month_QtySold']=QtySold
table_current_month ['Sell_Rate_Current_Month']=table_current_month.Current_Month_QtySold/table_current_month.monthlystock03
#The line above is changeable. monthlystock should change to current month
current_stock=Instock_import[['Item Description', 'Item No.', 'Store/Location Name', 'InStock']]

In [49]:
sell_rate_table_current_month=table_current_month[['Item Description', 'Item No.', 'Store/Location Name', 'Sell_Rate_Current_Month']].copy()

In [50]:
array=stock_and_sales_C[[i for i in stock_and_sales_C.columns if 'Sold' in i]].values

In [51]:
medianlast6months=np.median(array[:,-6:], axis=1)
medianlast3months=np.median(array[:,-3:], axis=1)
medianbeforelast6months = np.median (array[:,-6::-1])

In [52]:
stock_and_sales_D=stock_and_sales_C.copy()

In [53]:
stock_and_sales_D['units_sold_median_last6']=medianlast6months
stock_and_sales_D['units_sold_median_last3']=medianlast3months
stock_and_sales_D['units_sold_before_the_last6']=medianbeforelast6months

NOW THAT WE HAVE THE MIN MAX...WE CAN USE THE MEAN FROM MIN MAX AS HARMNONIZED MEAN

In [54]:
read_min_max= pd.read_excel("C:\\Users\\Geoffrey.Oute\\Desktop\\Automation Sandbox\\MIN MAX\\min max as at 04_03_2024.xlsx")

In [55]:
read_min_max['Mean'] = read_min_max.Mean.apply(lambda x: 1 if x < 1 else x)

In [56]:
read_min_max['Item No.'] = read_min_max['Item No.'].astype(str)

In [57]:
# read_min_max [read_min_max ['Item No.']==40225010242]

In [58]:
read_min_max.drop_duplicates(inplace=True) 

In [59]:
# nw=new_item_codes [['New Item No.', 'Existing Old Item Code']]
# nw=nw.copy()

In [60]:
# nw ['New Item No.'] = nw ['New Item No.'].astype(str) 
# nw ['Existing Old Item Code']  = nw ['Existing Old Item Code'].astype(str) 

In [61]:
def harmonizedmean(x):
    if x.units_sold_median_last6 <= x.units_sold_median_last3:
        return x.units_sold_median_last3
    
    elif x.units_sold_median_last6 <=0 and x.units_sold_median_last3 <=0:
        return medianbeforelast6months
        # in case we don't have last 6 months data, go back to before the last 6 months
    
    else:
        return x.units_sold_median_last6
    
harmonized_mean=stock_and_sales_D.apply(harmonizedmean, axis=1)

In [62]:
stock_and_sales_D['harmonized_mean'] = harmonized_mean

In [63]:
# stock_and_sales_D[stock_and_sales_D ['Item No.']=='30128680001'] [['Item No.', 'Store/Location Name', 'harmonized_mean']]

In [64]:
read_min_max.rename(columns= {'Branches': 'Store/Location Name'}, inplace=True)

In [65]:
read_min_max[read_min_max ['Item No.']==101000136]

,Item No.,Item Description,Department,Store/Location Name,selling_status,Last Seen,Mean,Main Order/Top Up Mean,MAX 1.5 Months


In [66]:
stock_and_sales_D=pd.merge(left=stock_and_sales_D, right = read_min_max[['Item No.', 'Store/Location Name', 'Mean']], on = ['Item No.','Store/Location Name'], how='left')

In [67]:
stock_and_sales_D ['Item No.'] = stock_and_sales_D ['Item No.'].astype(str)

In [68]:
# stock_and_sales_D [stock_and_sales_D ['Item No.']==18200910012].to_csv('sfdf.csv')

In [69]:
stock_and_sales_D.rename(columns={'harmonized_mean': 'old_harmonized_mean', 'Mean': 'harmonized_mean'}, inplace=True)

In [70]:
stock_and_sales_D['MaxStock']=stock_and_sales_D.harmonized_mean * 1.5
stock_and_sales_D['MinStock']=stock_and_sales_D.harmonized_mean * 0.9

In [71]:
stock_and_sales_D['Item No.'].info()

<class 'pandas.core.series.Series'>
Int64Index: 161740 entries, 0 to 161739
Series name: Item No.
Non-Null Count   Dtype 
--------------   ----- 
161740 non-null  object
dtypes: object(1)
memory usage: 2.5+ MB


In [72]:
current_stock.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102239 entries, 0 to 102238
Data columns (total 4 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Item Description     102239 non-null  object 
 1   Item No.             102239 non-null  object 
 2   Store/Location Name  102239 non-null  object 
 3   InStock              102239 non-null  float64
dtypes: float64(1), object(3)
memory usage: 3.1+ MB


In [73]:
current_stock ['Item No.'] = current_stock ['Item No.'].astype(str)

C:\Users\Geoffrey.Oute\AppData\Local\Temp\ipykernel_17320\2614597672.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current_stock ['Item No.'] = current_stock ['Item No.'].astype(str)


In [74]:
stock_and_sales_E=pd.merge(left=stock_and_sales_D, right=current_stock, on=['Item No.','Store/Location Name'], how='left')

In [75]:
stock_and_sales_E.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 161740 entries, 0 to 161739
Data columns (total 89 columns):
 #   Column                                 Non-Null Count   Dtype  
---  ------                                 --------------   -----  
 0   index                                  161740 non-null  int64  
 1   Item No.                               161740 non-null  object 
 2   Item Description_x                     161740 non-null  object 
 3   Brand                                  161740 non-null  object 
 4   Department                             161740 non-null  object 
 5   Location Type                          161740 non-null  object 
 6   Store/Location Name                    161740 non-null  object 
 7   Month_selling_frequency                161740 non-null  int64  
 8   Monthly_selling_frequency_last6months  161740 non-null  int64  
 9   Monthly_selling_frequency_last3months  161740 non-null  int64  
 10  Jan Qty Sold2023                       161740 non-null  

In [76]:
# nw['Item No.'].info()

### Sometimes, we have stock pushed to shops from upcountry warehouse. Lets add the stock to the instock of the shops so that we avoid overstocking.
This part of the code should be muted if there are no items being transferred from the upcountry warehouse at the time

### The code above should be deleted/muted if there is no item being transferred from upcountry warehouse

In [77]:
stock_and_sales_E=pd.merge(left=stock_and_sales_E, right=Tier1Shops, on=['Store/Location Name'], how='left')

In [78]:
stock_and_sales_E.InStock = stock_and_sales_E.InStock.fillna(0)

In [79]:
median_sell_rate=np.median(stock_and_sales_E[[i for i in stock_and_sales_E.columns if 'Sell' in i][-3:]],axis=1)

In [80]:
stock_and_sales_E['median_sell_rate']=median_sell_rate

In [81]:
sell_rate_table_current_month['Item No.'] = sell_rate_table_current_month['Item No.'].astype(str)
stock_and_sales_E['Item No.'] = stock_and_sales_E['Item No.'].astype(str)

In [82]:
sell_rate_table_current_month ['Item No.'] = sell_rate_table_current_month ['Item No.'].astype(str)

In [83]:
stock_and_sales_E=pd.merge(left=stock_and_sales_E, right=sell_rate_table_current_month, on=['Item No.', 'Store/Location Name'], how='left')

In [84]:
median_sell_rate_arrays=stock_and_sales_E.median_sell_rate.values
median_sell_rate_arrays[median_sell_rate_arrays==0]=0.01
#changing all median sell rates that are at 0 to 0.01 so that we can be able to calculate the months it will take to clear
#the stock

In [85]:
five_elements=stock_and_sales_E[stock_and_sales_E.isin({'Store/Location Name': ['RETAIL THE FIVE ELEMENTS', 'RETAIL FIVE ELEMENTS NYALI']})['Store/Location Name']]

#lets remove giftware shops from suggestion list

stock_and_sales_E.drop(five_elements.index, inplace=True)

In [86]:
def excess_stock_func(x):
    try:
        if x.Tier=='Tier1':
#             one_month_cover=x.InStock/(x.median_sell_rate*x.InStock)
            three_months_cover=x.harmonized_mean*3
            excess_stock=x.InStock-three_months_cover
        else:
#             one_month_cover=x.InStock/(x.median_sell_rate*x.InStock)
            two_months_cover=x.harmonized_mean*2
            excess_stock=x.InStock-two_months_cover
            
    except ZeroDivisionError:
        excess_stock = None
        
        
    return excess_stock    
        
        
        
        

In [87]:
def months_cover(x):
    try:
        if x.Tier=='Tier1':
#             one_month_cover=x.InStock/(x.median_sell_rate*x.InStock)
#lets use median qty sold only [harmonized]
            months_cover_2_3=x.harmonized_mean*3
           
        else:
#             one_month_cover=x.InStock/(x.median_sell_rate*x.InStock)
            months_cover_2_3=x.harmonized_mean*2
            
            
    except ZeroDivisionError:
        months_cover_2_3 = None
        
    return months_cover_2_3

In [88]:
excess_stock2=stock_and_sales_E.apply(excess_stock_func, axis=1)
months_cover=stock_and_sales_E.apply(months_cover, axis=1)

In [89]:
stock_and_sales_E['Months_Cover_2_3']=months_cover
stock_and_sales_E['Excess_stock']=excess_stock2

In [90]:
stock_and_sales_E[stock_and_sales_E ['Store/Location Name']=='RETAIL WAREHOUSE'].Excess_stock.unique()

array([nan])

CREATING THE MOVE TO TABLE
There are products in main warehouse that sell with a rate of below 0.25. e.g 11/100 a month. So for main warehouse, we pull it from the list and join it later so that we don't limit options that can move into main warehouse

In [91]:
stock_and_sales_F=stock_and_sales_E.copy()

In [92]:
main_warehouse_move_to=stock_and_sales_F[stock_and_sales_F['Store/Location Name'] == 'MAIN WAREHOUSE']

In [93]:
stock_and_sales_F.drop(main_warehouse_move_to.index, inplace=True)

In [94]:
# stock_and_sales_F[stock_and_sales_F['Item No.']=='162000018'][stock_and_sales_F[stock_and_sales_F['Item No.']=='162000018']['Store/Location Name']=='RETAIL SARIT']

In [217]:
move_to_table=stock_and_sales_F.copy()

In [218]:
move_to_table [move_to_table ['Item No.'] == '18200910012']

,index,Item No.,Item Description_x,Brand,Department,Location Type,Store/Location Name,Month_selling_frequency,Monthly_selling_frequency_last6months,Monthly_selling_frequency_last3months,...,MaxStock,MinStock,Item Description_y,InStock,Tier,median_sell_rate,Item Description,Sell_Rate_Current_Month,Months_Cover_2_3,Excess_stock


In [219]:
#On a second thought, lets not concatenate main. This will be taken care of by rebalancing  

In [220]:
move_to_table=pd.concat([move_to_table, 
#                          main_warehouse_move_to
                        ], 
                        ignore_index=True)

In [221]:
Golden_Life=move_to_table[move_to_table ['Store/Location Name'] ==  'RETAIL NAKURU GOLDENLIFE']

In [222]:
Elodret_Highlands = move_to_table[move_to_table ['Store/Location Name'] ==  'RETAIL ELDORET HIGHLANDS']

In [223]:
move_to_table.drop(Golden_Life.index, inplace=True)

In [224]:
move_to_table.drop(Elodret_Highlands.index, inplace=True)

In [225]:
move_to_table.loc[:, 'Capacity']=move_to_table.loc[:, 'MaxStock']-move_to_table.loc[:, 'InStock']

In [226]:
def check_improvement(x):
    if x.median_sell_rate <= 0.10 and x.Sell_Rate_Current_Month>=0.17:
        return 'Increased_Velocity'
    else:
        return 'Constant_Velocity'
    

In [227]:
# we have to omit receiving and production warehouse first. We'll rotate/send to of these categories later. First we deal with 
#what is in the shops. Order is shops, retail warehouse, main, then receiving and production

In [228]:
stock_and_sales_E.InStock.sum()

567332.41

In [229]:
pull_from_table=stock_and_sales_E[stock_and_sales_E['median_sell_rate']<0.25].copy()

#remember that table F does not have main warehouse. We dropped. But we can use table E
pull_from_table.sort_values(by='Excess_stock', ascending=False, inplace=True)
pull_from_table=pull_from_table[~pull_from_table.isin({'Store/Location Name': ['MAIN WAREHOUSE', 'PACKAGING MATERIALS WAREHOUSE','QUALITY CONTROL WAREHOUSE',
                                                              'RECEIVING WAREHOUSE', 'DAMAGES & EXPIRY WAREHOUSE','IVEND INTRANSIT WAREHOUSE',
                                                              'PRODUCTION WAREHOUSE', 'MAJID RETURNS WAREHOUSE', 'RETAIL OVERSTOCK WAREHOUSE',
                                                              'FIXED ASSETS & CONSUMMABLES','VARIANCE MAIN WAREHOUSE', 'RETAIL WAREHOUSE',
                                                             ]})['Store/Location Name']]

# pull_from_table= pull_from_table[pull_from_table.InStock>0]
# velocity_pl=pull_from_table.apply(check_improvement, axis=1)
# pull_from_table['Velocity']=velocity_pl
# pull_from_table=pull_from_table[pull_from_table ['Velocity'] == 'Constant_Velocity']
# pull_from_table['Unique_Index']=pull_from_table['Item Description_x']+pull_from_table['Store/Location Name']

In [230]:
pull_from_table_retail_warehouse=stock_and_sales_E[stock_and_sales_E['median_sell_rate']<0.25].copy()
#cant give retail warehouse median sell rate condition because it is not a selling warehouse
pull_from_table_retail_warehouse.sort_values(by='Excess_stock', ascending=False, inplace=True)
pull_from_table_retail_warehouse = pull_from_table_retail_warehouse[pull_from_table_retail_warehouse.isin({'Store/Location Name': ['RETAIL WAREHOUSE']})['Store/Location Name']]
pull_from_table_retail_warehouse= pull_from_table_retail_warehouse[pull_from_table_retail_warehouse.InStock>0]
velocity_pl_retail_warehouse=pull_from_table_retail_warehouse.apply(check_improvement, axis=1)
pull_from_table_retail_warehouse['Velocity']=velocity_pl_retail_warehouse
pull_from_table_retail_warehouse['Unique_Index'] = pull_from_table_retail_warehouse ['Item Description_x']+pull_from_table_retail_warehouse['Store/Location Name']

In [231]:
# pull_from_table_receiving_warehouse.Excess_stock.unique()

In [232]:
# #========================================================================================================================================================================================
pull_from_table_main_warehouse=stock_and_sales_E[stock_and_sales_E['median_sell_rate']<0.25].copy()
pull_from_table_main_warehouse.sort_values(by='Excess_stock', ascending=False, inplace=True)
pull_from_table_main_warehouse = pull_from_table_main_warehouse[pull_from_table_main_warehouse.isin({'Store/Location Name': ['MAIN WAREHOUSE']})['Store/Location Name']]
pull_from_table_main_warehouse= pull_from_table_main_warehouse[pull_from_table_main_warehouse.InStock>0]
velocity_pl_main_warehouse=pull_from_table_main_warehouse.apply(check_improvement, axis=1)
pull_from_table_main_warehouse['Velocity']=velocity_pl_main_warehouse
pull_from_table_main_warehouse ['Unique_Index'] = pull_from_table_main_warehouse ['Item Description_x']+ pull_from_table_main_warehouse ['Store/Location Name']
# #========================================================================================================================================================================================
pull_from_table_receiving_warehouse=stock_and_sales_E[stock_and_sales_E['median_sell_rate']<0.25].copy()
pull_from_table_receiving_warehouse.sort_values(by='Excess_stock', ascending=False, inplace=True)
pull_from_table_receiving_warehouse = pull_from_table_receiving_warehouse[pull_from_table_receiving_warehouse.isin({'Store/Location Name': ['RECEIVING WAREHOUSE']})['Store/Location Name']]
pull_from_table_receiving_warehouse= pull_from_table_receiving_warehouse[pull_from_table_receiving_warehouse.InStock>0]
velocity_pl_receiving_warehouse=pull_from_table_receiving_warehouse.apply(check_improvement, axis=1)
pull_from_table_receiving_warehouse['Velocity']=velocity_pl_receiving_warehouse
pull_from_table_receiving_warehouse ['Unique_Index'] = pull_from_table_receiving_warehouse ['Item Description_x'] + pull_from_table_receiving_warehouse ['Store/Location Name']
# #========================================================================================================================================================================================

pull_from_table_production_warehouse=stock_and_sales_E[stock_and_sales_E['median_sell_rate']<0.25].copy()
pull_from_table_production_warehouse.sort_values(by='Excess_stock', ascending=False, inplace=True)
pull_from_table_production_warehouse = pull_from_table_production_warehouse[pull_from_table_production_warehouse.isin({'Store/Location Name': ['PRODUCTION WAREHOUSE']})['Store/Location Name']]
pull_from_table_production_warehouse= pull_from_table_production_warehouse[pull_from_table_production_warehouse.InStock>0]
velocity_pl_production_warehouse=pull_from_table_production_warehouse.apply(check_improvement, axis=1)
pull_from_table_production_warehouse['Velocity']=velocity_pl_production_warehouse
pull_from_table_production_warehouse ['Unique_Index'] = pull_from_table_production_warehouse ['Item Description_x']+ pull_from_table_production_warehouse ['Store/Location Name']                                                  
                                        

In [233]:
#Ignore pull_from_table cos this will pull from other shops
#Lets not pull production whse
# We'll remove the two above from concat

In [234]:
combined_table=pd.concat([pull_from_table_retail_warehouse, pull_from_table_main_warehouse, pull_from_table_receiving_warehouse])

In [235]:
def Capactiy2(x):
    if x.Capacity > 0 and x.Capacity<1:
        return 1
    elif x.Capacity >= 1:
        return round(x.Capacity)
    else:
        return x.Capacity

In [236]:
stock_and_sales_E [stock_and_sales_E['Item No.']=='149000057']

,index,Item No.,Item Description_x,Brand,Department,Location Type,Store/Location Name,Month_selling_frequency,Monthly_selling_frequency_last6months,Monthly_selling_frequency_last3months,...,MaxStock,MinStock,Item Description_y,InStock,Tier,median_sell_rate,Item Description,Sell_Rate_Current_Month,Months_Cover_2_3,Excess_stock
90881,96579,149000057,NATURALLI CAJUN SPICE 100G,NATURALLI,Herbs & Spices,Warehouse,DAMAGES & EXPIRY WAREHOUSE,0,0,0,...,NaN,NaN,NaN,0.0,NaN,0.01,NaN,NaN,NaN,NaN
90882,96580,149000057,NATURALLI CAJUN SPICE 100G,NATURALLI,Herbs & Spices,Warehouse,IVEND INTRANSIT WAREHOUSE,0,0,0,...,NaN,NaN,NaN,0.0,NaN,0.01,NaN,NaN,NaN,NaN
90883,96581,149000057,NATURALLI CAJUN SPICE 100G,NATURALLI,Herbs & Spices,Warehouse,MAIN WAREHOUSE,0,0,0,...,1.5000,0.9000,NATURALLI CAJUN SPICE 100G,1.0,NaN,0.01,NATURALLI CAJUN SPICE 100G,0.0,2.000,-1.000
90884,96582,149000057,NATURALLI CAJUN SPICE 100G,NATURALLI,Herbs & Spices,Retail,RETAIL ABC,1,1,1,...,1.5000,0.9000,NaN,0.0,NaN,0.01,NaN,NaN,2.000,-2.000
90885,96583,149000057,NATURALLI CAJUN SPICE 100G,NATURALLI,Herbs & Spices,Retail,RETAIL CAPITAL CENTRE,2,2,2,...,1.5000,0.9000,NaN,0.0,NaN,0.01,NaN,NaN,2.000,-2.000
90886,96584,149000057,NATURALLI CAJUN SPICE 100G,NATURALLI,Herbs & Spices,Retail,RETAIL ELDORET,0,0,0,...,1.5000,0.9000,NaN,0.0,NaN,0.01,NaN,NaN,2.000,-2.000
90887,96585,149000057,NATURALLI CAJUN SPICE 100G,NATURALLI,Herbs & Spices,Retail,RETAIL GALLERIA,3,3,3,...,1.5000,0.9000,NaN,0.0,NaN,0.01,NaN,NaN,2.000,-2.000
90888,96586,149000057,NATURALLI CAJUN SPICE 100G,NATURALLI,Herbs & Spices,Retail,RETAIL GARDEN CITY,1,1,0,...,1.5000,0.9000,NaN,0.0,NaN,0.01,NaN,NaN,2.000,-2.000
90889,96587,149000057,NATURALLI CAJUN SPICE 100G,NATURALLI,Herbs & Spices,Retail,RETAIL HILTON,2,2,2,...,1.5000,0.9000,NATURALLI CAJUN SPICE 100G,2.0,Tier1,0.01,NATURALLI CAJUN SPICE 100G,0.0,3.000,-1.000
90890,96588,149000057,NATURALLI CAJUN SPICE 100G,NATURALLI,Herbs & Spices,Retail,RETAIL IMAARA MALL,0,0,0,...,1.5000,0.9000,NaN,0.0,NaN,0.01,NaN,NaN,2.000,-2.000


In [237]:
move_to_table.insert(6, 'Vendor Name', 'N/A')

In [238]:
move_to_table_B=move_to_table[[ 'Item No.','Item Description_x','Brand', 'Store/Location Name', 'Location Type',
                                                     'Vendor Name', 'median_sell_rate','MaxStock', 'MinStock','Capacity','InStock'
              
                                                     ]].copy()



move_to_table_B['CapacityRounded']=move_to_table_B.apply(Capactiy2, axis=1)

In [239]:
giftware=move_to_table_B[move_to_table_B.isin({'Store/Location Name': ['RETAIL THE FIVE ELEMENTS', 'RETAIL FIVE ELEMENTS NYALI']})['Store/Location Name']]

#lets remove giftware shops from suggestion list

move_to_table_B.drop(giftware.index, inplace=True)

In [240]:
combined_table2=pd.concat([Instock_import[Instock_import ['Store/Location Name'] =='RETAIL WAREHOUSE'].copy(),
                           Instock_import[Instock_import ['Store/Location Name'] =='MAIN WAREHOUSE'].copy(),
                           
                          ])

In [241]:
combined_table2 ['Unique_Index'] = combined_table2 ['Item Description'] + combined_table2 ['Store/Location Name']

In [242]:
combined_table2.rename(columns={'InStock': 'Excess_stock', 'Item Description': 'Item Description_x'}, inplace=True)

In [243]:
# move_to_table[move_to_table ['Item No.']=='155000324'].to_csv('sdsf.csv')

In [244]:
read_min_max ['Item No.'] = read_min_max ['Item No.'].astype(str)

combined_table2 ['Item No.'] = combined_table2 ['Item No.'].astype(str)

In [245]:
combined_table3=pd.merge(left=combined_table2, right = read_min_max [['Item No.', 'Store/Location Name', 'Mean']], on =['Item No.', 'Store/Location Name'], how='left')

if Item has recently picked sales, above or equal to 0.2, leave the stock as is. Else remove excess.

In [246]:
combined_table3.columns

Index(['#', 'Mode of Business', 'Core Type', 'Location Type', 'Item No.',
       'Bar Code', 'Item Description_x', 'Department', 'Brand', 'Vendor Code',
       'Vendor Name', 'Store/Location Code', 'Store/Location Name',
       'Excess_stock', 'Opening Stock', 'Purchased Quantity',
       'Purchase Return Quantity', 'Open PO Qty', 'Order Quantity',
       'Jan Qty Sold', 'Feb Qty Sold', 'Mar Qty Sold', 'Apr Qty Sold',
       'May Qty Sold', 'Jun Qty Sold', 'Jul Qty Sold', 'Aug Qty Sold',
       'Sep Qty Sold', 'Oct Qty Sold', 'Nov Qty Sold', 'Dec Qty Sold',
       'Total Quantity Sold', 'Closing Stock', 'Retail Price Ex. VAT',
       'Retail Price Inc VAT', 'COGS', 'Total Sales Amount', 'Gross Profit',
       'Gross Profit %', 'Unique_Index', 'Mean'],
      dtype='object')

RECENT SHIPMENTS

In [247]:
# HW1= pd.read_excel("C:\\Users\\Geoffrey.Oute\\Desktop\\Recent Shipments\\HW1.xlsx")

In [248]:
# HW2= pd.read_excel("C:\\Users\\Geoffrey.Oute\\Desktop\\Recent Shipments\\HW2.xlsx")

In [249]:
# all_items_HW=list(HW1 ['Item Num'].unique())+ list(HW2 ['Item Num'].unique())

In [250]:
# combined_table4=combined_table3 [~combined_table3.isin ({'Item No.': all_items_HW}) ['Item No.']]

In [251]:
combined_table4 = combined_table3.copy()

In [252]:
def Brand_Block(x):
    if (x['Brand'] == 'ECOVER' or x ['Brand'] == 'METHOD') and (x ['Store/Location Name'] in ['RETAIL HILTON', 'RETAIL T-MALL', 'RETAIL CAPITAL CENTRE', 'RETAIL EASTLEIGH BRANCH', 'RETAIL MAIYAN MALL']):
        return 'Block'
    else:
        return 'Proceed'

In [253]:
def Brand_Block2(x):
    if (x['Brand'] == "DR BRONNER'S" or x ['Brand'] == 'HEMANI') and (x ['Store/Location Name'] in ["CARREFOUR MEGA (EXPRESS)"]):
        return 'Block'
    else:
        return 'Proceed'

In [254]:
move_to_table_B ['Block_Status'] = move_to_table_B.apply(Brand_Block2, axis=1)

In [255]:
move_to_table_B ['Block_Status'] = move_to_table_B.apply(Brand_Block, axis=1)

In [256]:
department_dictionary=dict(read_min_max[['Item No.', 'Department']].values)

In [257]:
move_to_table_B ['Department'] = move_to_table_B['Item No.'].apply(lambda x: department_dictionary[x] if x in department_dictionary else 'No Dep')

In [258]:
def Department_Block(x):
    if (x['Department'] == 'Baby foods') and (x ['Store/Location Name'] in ['RETAIL HILTON']):
        return 'Block'
    else:
        return 'Proceed'
    

In [259]:
move_to_table_B ['Department_Block'] = move_to_table_B.apply(Department_Block, axis=1)

In [260]:
indices_omit=list(set(list(move_to_table_B [move_to_table_B ['Block_Status'] == 'Block'].index)+list(move_to_table_B [move_to_table_B ['Department_Block'] == 'Block'].index)))

In [261]:
move_to_table_B.drop(indices_omit, inplace=True)

In [262]:
move_to_table_B.set_axis(range(0, len(move_to_table_B)), inplace=True)

In [263]:
move_to_table_B.drop(['Block_Status', 'Department', 'Department_Block'], axis=1, inplace=True)

In [264]:
move_to_table_B['Store/Location Name'].unique()

array(['IVEND INTRANSIT WAREHOUSE', 'RETAIL ABC', 'RETAIL CAPITAL CENTRE',
       'RETAIL EASTLEIGH BRANCH', 'RETAIL ELDORET', 'RETAIL GALLERIA',
       'RETAIL GARDEN CITY', 'RETAIL HILTON', 'RETAIL IMAARA MALL',
       'RETAIL JUNCTION', 'RETAIL KISUMU', 'RETAIL KISUMU MEGA',
       'RETAIL LAVINGTON', 'RETAIL LIKONI', 'RETAIL MAIYAN MALL',
       'RETAIL NAKURU', 'RETAIL NANYUKI', 'RETAIL NYALI',
       'RETAIL NYALI CENTRE', 'RETAIL REHANI', 'RETAIL RHAPTA ROAD',
       'RETAIL RIVERSIDE SQUARE', 'RETAIL SARIT', 'RETAIL T-MALL',
       'RETAIL THE HUB', 'RETAIL THIKA ROAD MALL', 'RETAIL TWO RIVERS',
       'RETAIL VILLAGE MARKET', 'RETAIL WESTGATE',
       'RETAIL WESTLANDS SQUARE', 'RETAIL YAYA', 'UPCOUNTRY WAREHOUSE',
       'RETAIL NAIVASHA', 'RETAIL THE PLACE 67', 'RETAIL WAREHOUSE',
       'DAMAGES & EXPIRY WAREHOUSE', 'RECEIVING WAREHOUSE',
       'PRODUCTION WAREHOUSE', 'QUALITY CONTROL WAREHOUSE',
       'RETAIL OVERSTOCK WAREHOUSE', 'X WAREHOUSE',
       'MAJID RETURNS WAR

In [265]:
move_to_table_B = move_to_table_B [move_to_table_B .isin({'Store/Location Name': ['RETAIL SARIT']})['Store/Location Name']]

In [266]:
# move_to_table_B = move_to_table_B [~move_to_table_B .isin({'Store/Location Name': ['RETAIL SARIT','RETAIL YAYA', 'RETAIL VILLAGE MARKET', 'RETAIL WESTGATE', 'RETAIL JUNCTION', 'RETAIL NYALI', 'RETAIL NAKURU', 'RETAIL HILTON']})['Store/Location Name']]

In [267]:
move_to_table_B ['Store/Location Name'].unique()

array(['RETAIL SARIT'], dtype=object)

In [268]:
move_to_table_B.head(2)

,Item No.,Item Description_x,Brand,Store/Location Name,Location Type,Vendor Name,median_sell_rate,MaxStock,MinStock,Capacity,InStock,CapacityRounded
22,159000001,A.VOGEL AGNUS CASTUS 50ML,A.VOGEL,RETAIL SARIT,Retail,N/A,0.01,1.5,0.9,-4.5,6.0,-4.5
54,159000002,A.VOGEL ECHINAFORCE 120TABS,A.VOGEL,RETAIL SARIT,Retail,N/A,0.01,1.5,0.9,1.5,0.0,2.0


In [269]:
blocked_items ['Status'] = 'Block'

In [270]:
blocked_items['Item No.'] = blocked_items['Item No.'].astype(str)

In [271]:
move_to_table_B ['Item_Shop'] = move_to_table_B ['Item No.']+ move_to_table_B ['Store/Location Name']

In [272]:
blocked_items ['Item_Shop'] =blocked_items ['Item No.']+ blocked_items ['Retail Shop']

In [273]:
blocked_items.rename(columns={'Item No.': 'Item Code'}, inplace=True)

In [274]:
move_to_table_BBB = pd.merge(left = move_to_table_B, right = blocked_items, on=['Item_Shop'], how='left')

In [275]:
move_to_table_B = move_to_table_BBB [~move_to_table_BBB.isin({'Status': ['Block']})['Status']].copy()

In [276]:
# move_to_table_B.drop(['Item_Shop', 'Item '])

In [277]:
move_to_table_B.drop(move_to_table_B.iloc[:, 12:].columns, axis=1, inplace=True)

In [278]:
rotation =[]
rotation2 =[]

In [279]:
combined_table4.Excess_stock

0       101.0
1        26.0
2         8.0
3        32.0
4        30.0
        ...  
2106      0.0
2107      1.0
2108      0.0
2109      0.0
2110      0.0
Name: Excess_stock, Length: 2111, dtype: float64

In [280]:
def FilterTable():
    for i in combined_table4.Unique_Index.values:
        temp_table=combined_table4[combined_table4 ['Unique_Index'] == i]
        brand=temp_table.Brand.values[0]
        department = temp_table.Department.values[0]
        
        units_aged=temp_table.Excess_stock.values[0] #change this part..units_aged should be excess_stock. code is from ageing stock rotation
        item_code = temp_table['Item No.'].values[0]
        item_desc=temp_table['Item Description_x'].values[0]
        from_shop=temp_table['Store/Location Name'].values[0]
        
        temp_table_move_to=move_to_table_B[move_to_table_B['Item No.']==item_code].sort_values(by='Capacity', 
                                                                                         ascending=False).sort_values(by='InStock',
                                                                                                                      ascending=True)
        #let go of item description. It changes. Use Item No.
        try:
            if temp_table_move_to.iloc[:1]['Store/Location Name'].values[0]=='MAIN WAREHOUSE':
                delete=temp_table_move_to[:1]
                temp_table_move_to.drop(delete.index, inplace=True)
                temp_table_move_to=pd.concat([temp_table_move_to, delete])
        except:
            IndexError
 
        
        for i in temp_table_move_to.itertuples(name=None):
            if i[12]>0 and units_aged>=i[12]:
                can_take=i[12]
                to_store=i[4]
                in_stock_to = i[11]
                max_stock_to=i[8]
                
                rotation.append([item_code,item_desc,from_shop,to_store,in_stock_to,max_stock_to,can_take,brand, department])
                units_aged=units_aged - can_take
            
                move_to_table_B.loc[i[0]:i[0], 'CapacityRounded']=move_to_table_B.loc[i[0]:i[0], 'CapacityRounded'] - can_take
            
                if units_aged<=0:
                    break;
            elif i[12]>0 and units_aged<i[12]:
                can_take=units_aged
                to_store=i[4]
                in_stock_to = i[11]
                max_stock_to=i[8]
                
                rotation.append([item_code,item_desc,from_shop,to_store,in_stock_to,max_stock_to,can_take, brand, department])
                
                units_aged=units_aged - can_take
                
                move_to_table_B.loc[i[0]:i[0], 'CapacityRounded']=move_to_table_B.loc[i[0]:i[0], 'CapacityRounded'] - can_take
                if units_aged<=0:
                    break;
                    
        if units_aged>0:
            other_options = 'other_options'
            to_storeB=from_shop
            rotation2.append([item_code,item_desc ,from_shop, to_storeB,units_aged,'N/A',other_options, brand, department])
                
    table1= pd.DataFrame(rotation+rotation2, columns=['Item Code','ItemDescription' ,'MoveFrom', 'MoveTo', 'Instock', 'MaxStock', 'Movable_Units', 'Brand', 'Department'])
    return table1
    

In [281]:
tableP=FilterTable()

In [282]:
tableQ=tableP.copy()
other_options=tableQ[tableQ.Movable_Units=='other_options']
tableQ.drop(other_options.index, inplace=True)
tableQ=tableQ[tableQ.Movable_Units>0]

In [283]:
less_than_30 = tableQ[tableQ ['Instock'] < 200]

In [284]:
less_than_30=less_than_30.sort_values(by='Instock')

In [285]:
# move_to_table_B [move_to_table_B ['Item No.']=='14200720001']

In [286]:
# less_than_30 [less_than_30 ['Item Code']=='14200720001']

In [288]:
ivend2 = pd.read_excel("C:\\Users\\Geoffrey.Oute\\Desktop\\Automation Sandbox\\Positive Inventory\\Positive Inventory (Retail) 07.03.2024.xlsx")
ivend_comb=pd.concat([ivend2['Item No.'], ivend2[[i for i in ivend2.columns if 'IVEND' in i]]], axis=1)
cols_extracted = ['RETAIL' + " " + i.replace('IVEND INTRANSIT', "").strip() for i in ivend_comb.columns if 'IVEND' in i]
ivend_comb.columns = ['Item No.'] + cols_extracted
ivend_temp1=ivend_comb [ivend_comb.iloc[:, 1:].sum(axis=1)>0].copy()

In [289]:
holding_list=[]

In [290]:
for i in ivend_temp1.columns[1:]:
    
    inner_temp = ivend_temp1[['Item No.',i]][ivend_temp1[['Item No.',i]][i]>0]

    inner_temp ['To Warehouse'] = inner_temp.columns[1]

    inner_temp.groupby(['Item No.', 'To Warehouse']).sum().reset_index(inplace=True)

    holding_list.append(inner_temp.values)

In [291]:
ivend_final = pd.concat([pd.DataFrame(i) for i in holding_list], ignore_index=True)

ivend_final.columns = ['Product Code', 'UOM Quantity Open', 'To Warehouse']

In [292]:
ivend_final['Product Code'] = ivend_final['Product Code'].astype(int)

In [293]:
ivend_final.columns = ['Item Code', 'Ivend_Open', 'MoveTo']

In [294]:
ivend_current = ivend_final[['Item Code', 'MoveTo', 'Ivend_Open']].copy()

In [295]:
ivend_current['Item Code'] = ivend_current['Item Code'].astype(str)

In [296]:
less_than_30.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 378 entries, 519 to 132
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Item Code        378 non-null    object 
 1   ItemDescription  378 non-null    object 
 2   MoveFrom         378 non-null    object 
 3   MoveTo           378 non-null    object 
 4   Instock          378 non-null    float64
 5   MaxStock         378 non-null    object 
 6   Movable_Units    378 non-null    object 
 7   Brand            378 non-null    object 
 8   Department       378 non-null    object 
dtypes: float64(1), object(8)
memory usage: 29.5+ KB


In [297]:
merged=pd.merge(left=less_than_30, right=ivend_current, on = ['Item Code', 'MoveTo'], how='left')

merged.Ivend_Open.fillna(0, inplace=True)

In [298]:
def subtract_ivend(x):
    if x.Ivend_Open == 0:
        return x.Movable_Units
    
    elif (x.Ivend_Open + x.Instock) > x.MaxStock:
    
        return 'Max Stock Exceeded'
    
    elif (x.Ivend_Open + x.Instock) < x.MaxStock:
        
        return x.MaxStock - (x.Ivend_Open + x.Instock)
        
    else:
        return x.Movable_Units

In [299]:
merged=less_than_30.copy()

In [300]:
merged.rename(columns={'Movable_Units': 'Units'}, inplace=True)

In [301]:
# merged ['Units'] = iven_subtracted

In [302]:
# merged[merged ['Item Code']==14304380005]

In [303]:
finalXXX=merged[['Item Code', 'ItemDescription', 'MoveFrom', 'MoveTo', 'Units', 'Instock',
       'MaxStock', 'Brand', 'Department']].copy()

In [304]:
finalXXX.rename(columns= {'MoveFrom': 'TransferFrom', 'MoveTo': 'TransferTo'}, inplace=True)

In [305]:
finalXXX.head(2)

,Item Code,ItemDescription,TransferFrom,TransferTo,Units,Instock,MaxStock,Brand,Department
519,165000116,NATURALLI JAPAN RICE CRACKER MIX 100G,MAIN WAREHOUSE,RETAIL SARIT,4.0,0.0,10.10625,NATURALLI,Naturalli
323,143000017,ALPRO OAT DRINK UNSWEETENED 1LTR#80006233,MAIN WAREHOUSE,RETAIL SARIT,12.0,0.0,73.40625,ALPRO,Dairy Alternatives


In [306]:
# def units_adj(x):
#     if x.Units < 4:
#         return 4
#     else:
#         return math.ceil(x.Units)

In [307]:
#  dont_supply[dont_supply ['Item Code']==14304380001]

In [308]:
dont_supply=finalXXX[finalXXX.Units == 'Max Stock Exceeded'] 

In [309]:
finalXXX.drop(dont_supply.index, inplace=True)

In [310]:
finalXXX[finalXXX ['Item Code']=='14304380005'].head(3)

,Item Code,ItemDescription,TransferFrom,TransferTo,Units,Instock,MaxStock,Brand,Department


In [311]:
# finalXXX['Units_'] = finalXXX.apply(units_adj, axis=1)

In [312]:
finalXXX.rename(columns={'Units': 'Units_'}, inplace=True)

In [313]:
finalXXX = finalXXX [['Item Code', 'ItemDescription', 'TransferFrom', 'TransferTo', 'Units_',
       'Instock', 'MaxStock', 'Brand', 'Department']]

In [314]:
finalXXX = finalXXX [['Item Code', 'ItemDescription', 'TransferFrom', 'TransferTo', 'Units_',
       'Instock', 'MaxStock', 'Brand', 'Department']]

In [315]:
finalXXX ["Manager's Feedback"] = " "

In [316]:
finalXXX = finalXXX[finalXXX ['Units_']>0]

In [317]:
finalXXX['Item Code'].astype(str)

519    165000116
323    143000017
84     159000926
330    159001205
339    157000201
         ...    
30     159000824
223    162000257
317    143000008
20     143000008
132    162000175
Name: Item Code, Length: 378, dtype: object

#### Excluding recent shipments!!!

Activate To Code When There is New Shipment


In [318]:
# bulk.to_csv('bulk.csv')

In [319]:
# non_bulk.to_csv('non_bulk_10_07.csv')

Mute the line below if there is shipment. If there are no shipments, activate the line.

In [320]:
finalAAA=finalXXX[~finalXXX.isin({'Department': ['Frozen Foods']}) ['Department']].copy()

In [321]:
finalAAA.rename(columns={'Units': 'Units_'}, inplace=True)

THIS PART OF THE CODE SHOULD ALWAYS BE THERE!!!!

In [322]:
def reduction_large_containers(x):
    if (x.Brand == 'MUSCLETECH' or x.Brand == 'OPTIMUM NUTRITION' or x.Brand == 'RULE 1') and x.Instock>=2:
        return 0

    elif (x.Brand == 'MUSCLETECH' or x.Brand == 'OPTIMUM NUTRITION' or x.Brand == 'RULE 1') and ('LB' in x.ItemDescription or 'KG' in x.ItemDescription) and x.Units_>5 and (x.TransferTo == 'RETAIL HILTON' or x.TransferTo == 'RETAIL SARIT' or x.TransferTo == 'RETAIL YAYA' or x.TransferTo == 'RETAIL WESTGATE' ):
        return 2
    
    elif (x.Brand == 'MUSCLETECH' or x.Brand == 'OPTIMUM NUTRITION' or x.Brand == 'RULE 1') and ('LB' in x.ItemDescription or 'KG' in x.ItemDescription) and x.Units_>=3:
        return 1
    
    elif (x.Brand == 'MUSCLETECH' or x.Brand == 'OPTIMUM NUTRITION' or x.Brand == 'RULE 1') and x.Units_>=3:
        return 2
    else:
        return x.Units_

In [323]:
units_AAA=finalAAA.apply(reduction_large_containers, axis=1)

In [324]:
finalAAA ['Units_']=units_AAA

In [325]:
finalAAA.drop_duplicates(inplace=True)

In [339]:
finalCCC=finalAAA[finalAAA.isin({'Department': ['Bulk Foods', 'Live Life Well', 'Naturalli', 'Herbs & Spices']})['Department']].copy()

In [340]:
finalCCC = finalCCC[~finalCCC.isin({'Brand': ["THOMPSON", 'LURON MASSAGE OILS', 'JANI', 'PIKSTERS','ESSENZA', 'HYPODEA','BUKE', 'CINNABAR', 'NIMBLE', 'CLEO NATURE','HELIOCARE 360*', 'BIOFREEZE', 'DIETERS', "O'BAO", 'YVES ROCHER', 'FUSHI', 'ORGANIC SURGE', 'BOOCH', 'GROUNDED','CHIA ORGANIC', 'HGC', 'KAREMBOU SPA', 'KEOLA', 'COTONEVE', 'FAMACOTONE', 'KOKO COSMETICS', 'COTTONS', 'SEVEN SEAS','BIO BALANCE', 'SHEA MOISTURE', 'LULU', 'BREATH OF AFRICA', 'MARINI NATURALS', 'MUSK CARE', 'FLINTSHINE', 'NANO','NATURES WAY', 'UNWIND', 'OPES', 'BOOKS/CARDS', 'WHEY2GO', 'K2U', 'RCI', 'ENSURE', 'BIW', 'IGNITE', 'ANTHEM','BIO-OIL', 'USN', 'PENTA SURE', 'DUNGA SPIRULINA', 'TIWANI', 'UNCOVER', 'VAAJI', 'DIATOMACEOUS', 'SOYBIO', 'KAYA NUTS','ROYAL UMRELLA', 'BAKULI FOODS', 'CHOCODATE', 'BARAKA ISRAEL', 'BIO FOODS', 'BLACK JUICE', 'BERNARD', 'SWEETUNDA', 'BUTTYBOX', 'CAROLINE CUPCAKES', 'BORGES', 'SASANI', 'ELI ORGANIC', 'GABBY & KASS','GRACE CHOCOLATERIE', 'KAJUJU', 'AKAGI', 'IWAI', 'KIKKOMAN', 'NIPPN', 'KEVAN', 'APTMAIL', 'HUGGIES','MAYERS', 'MTOFU', 'MTINDI', 'NUTRI DELIGHTS', 'OFELOS', 'PLANTA', 'PURE & SURE', 'CADBURY', 'MOHAWK', 'SEVEN DAY', 'ZAWADI', 'SPRING VALLEY COFFEE', 'ASILI AFRICA', 'SUNLAM', 'OSTERBERG','MENTOS', 'MCVITIES', 'TAGO', 'MORE THAN']})['Brand']]

In [341]:
finalDDD=finalCCC[finalCCC.Units_>0].copy()

In [342]:
finalDDD = finalDDD[~finalDDD.isin({'Department': ['M/harvest']})['Department']]

In [343]:
finalDDD ['Units_']= round(finalDDD ['Units_'])

In [348]:
destination_path = "C:\\Users\\Geoffrey.Oute\\Desktop\\Automation Sandbox\\Top Ups_1\\07_03_2024 Groups A SARIT Bulk\\"

In [349]:
# finalDDD.drop('MaxStock', axis=1, inplace=True)

In [350]:
todaysdate= '07_03_24'

In [351]:
# final

In [352]:
# finalDDD.drop('MaxStock', axis=1,inplace=True)

In [353]:
finalEEE =finalDDD[~finalDDD.isin({'ItemDescription': ['GARDEN OF LIFE GRASS FED COLLAGEN PEPTIDES (SACHET) 20G#103535']})['ItemDescription']].copy()

In [354]:
finalFFF = finalEEE[finalEEE.Units_ > 0].copy()

#This line gets rid of items in ivend. They will be reading 0

In [355]:
for i in finalFFF.TransferTo.unique():
    temp=finalFFF[finalFFF.loc[:, 'TransferTo'] == i]
# #     temp.reindex()
#     temp2_bulk = bulk_movable[bulk_movable.loc[:, 'MoveFrom']==i]
# #     temp2_bulk.reindex()
    
    with pd.ExcelWriter (destination_path+i+' '+todaysdate+'.xlsx') as writer:
        temp.to_excel(writer, sheet_name='Top Up_Main Order', index=False)
#         temp2_bulk.to_excel(writer, sheet_name = 'Bulk_Items' ,index=False)
        
        
#     temp.to_excel(path+i+'.csv' ,index=False)|
    
#      with pd.ExcelWriter('output.xlsx') as writer:  # doctest: +SKIP
# ...     df1.to_excel(writer, sheet_name='Sheet_name_1')
# ...     df2.to_excel(writer, sheet_name='Sheet_name_2')
    

In [ ]:
finalFFF.to_csv("C:\\Users\\Geoffrey.Oute\\Desktop\\Automation Sandbox\\Top Ups_1\\12_02_2024 Groups B_C\\Groups B_C 12_02_2024.csv", index=False)

In [ ]:
destination_path_bulk = "C:\\Users\\Geoffrey.Oute\\Desktop\\Automation Sandbox\\Top Ups_1\\12_02_2024 Groups B_C Bulk, Live Life, Naturalli, Herbs_Spices\\"

In [ ]:
# finalDDD.drop('MaxStock', axis=1, inplace=True)

In [ ]:
# finalDDD.TransferTo.unique()

In [ ]:
for i in finalFFF.TransferTo.unique():
    temp=finalFFF[finalFFF.loc[:, 'TransferTo'] == i]
# #     temp.reindex()
#     temp2_bulk = bulk_movable[bulk_movable.loc[:, 'MoveFrom']==i]
# #     temp2_bulk.reindex()
    
    with pd.ExcelWriter (destination_path_bulk+i+' '+'BULK'+ ' ' +todaysdate+'.xlsx') as writer:
        temp.to_excel(writer, sheet_name='Bulk Top Up_Main Order', index=False)
#         temp2_bulk.to_excel(writer, sheet_name = 'Bulk_Items' ,index=False)
        
        
#     temp.to_excel(path+i+'.csv' ,index=False)
    
#      with pd.ExcelWriter('output.xlsx') as writer:  # doctest: +SKIP
# ...     df1.to_excel(writer, sheet_name='Sheet_name_1')
# ...     df2.to_excel(writer, sheet_name='Sheet_name_2')

In [ ]:
finalFFF.to_csv("C:\\Users\\Geoffrey.Oute\\Desktop\\Automation Sandbox\\Top Ups_1\\12_02_2024 Groups B_C Bulk, Live Life, Naturalli, Herbs_Spices\\bulkmainorderGroupsB_C12_02_2024.csv", index=False)

In [ ]:
combined_table4 [combined_table4 ['Item No.']==31823300006]